<a href="https://colab.research.google.com/github/MohammadEmon01/Real-time-Language-Translation-using-Neural-Machine-Translation/blob/main/LanguageTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deep_translator gradio transformers sentencepiece torch --quiet

import torch
import gradio as gr
from transformers import MarianMTModel, MarianTokenizer
from deep_translator import GoogleTranslator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.6 MB/s eta 0:00:00


In [2]:
!pip install --upgrade optree

In [3]:
# Language option mapping
# -------------------------------
Languages = {
    "English": "en",
    "Arabic": "ar",
    "Bengali": "bn",
    "Hindi": "hi",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Italian": "it",
    "Japanese": "ja",
}

In [4]:
# Load translation model safely
# -------------------------------
def load_model(src_lang, tgt_lang):
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}"
    try:
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)
        return tokenizer, model
    except Exception:
        return None, None


In [5]:
# Translation logic
# -------------------------------
def translate(text, src_lang_name, tgt_lang_name):
    if not text:
        return " Please enter text to translate."

    src_lang = Languages.get(src_lang_name)
    tgt_lang = Languages.get(tgt_lang_name)

    if not src_lang or not tgt_lang:
        return " Error: Invalid language names."

    # Try loading model
    tokenizer, model = load_model(src_lang, tgt_lang)

    # If model not available, fallback to GoogleTranslator
    if tokenizer is None or model is None:
        try:
            translated = GoogleTranslator(source=src_lang, target=tgt_lang).translate(text)
            return f"(GoogleTranslator)\n{translated}"
        except Exception as e:
            return f" Error using GoogleTranslator: {str(e)}"

    # Run translation with MarianMT
    try:
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = model.generate(**inputs)
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translated_text
    except Exception as e:
        return f" Error: {str(e)} (translation failed)"

# -------------------------------
# Clear fields
# -------------------------------
def clear_field():
    return "", "English", "Bengali", ""



In [6]:
# Gradio UI
# -------------------------------
with gr.Blocks() as app:
    gr.Markdown("<h1 style='text-align:center'> Language Translator</h1>")
    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(label="Enter Text")
            src_lang_radio = gr.Radio(
                label="Select Source Language",
                choices=list(Languages.keys()),
                value="English"
            )
            tgt_lang_radio = gr.Radio(
                label="Select Target Language",
                choices=list(Languages.keys()),
                value="Bengali"
            )
            with gr.Row():
                translate_button = gr.Button("Translate")
                clear_button = gr.Button("Clear")
        with gr.Column():
            output_text = gr.Textbox(label="Translated Text", interactive=False)

    # Button click actions
    translate_button.click(
        translate,
        inputs=[text_input, src_lang_radio, tgt_lang_radio],
        outputs=output_text
    )

    clear_button.click(
        clear_field,
        inputs=[],
        outputs=[text_input, src_lang_radio, tgt_lang_radio, output_text]
    )

app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5db91b01f2217abfbd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
